<a href="https://colab.research.google.com/github/Rizzshen/AiGoogleCollab/blob/main/Worksheet8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Worksheet 8: Decision Tree, Ensemble Methods & Hyperparameter **Tuning**

## 1. Import Required Libraries

In [2]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris, load_wine
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error


##2. Custom Decision Tree (Using Information Gain)

In [1]:
class CustomDecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _entropy(self, y):
        probs = np.bincount(y) / len(y)
        return -np.sum(probs * np.log2(probs + 1e-9))

    def _information_gain(self, parent, left, right):
        weight_l = len(left) / len(parent)
        weight_r = len(right) / len(parent)
        return self._entropy(parent) - (weight_l * self._entropy(left) + weight_r * self._entropy(right))

    def _build_tree(self, X, y, depth=0):
        if len(np.unique(y)) == 1 or (self.max_depth and depth >= self.max_depth):
            return {'class': np.bincount(y).argmax()}

        best_gain = -1
        best_split = None

        for feature in range(X.shape[1]):
            for threshold in np.unique(X[:, feature]):
                left = y[X[:, feature] <= threshold]
                right = y[X[:, feature] > threshold]
                if len(left) == 0 or len(right) == 0:
                    continue
                gain = self._information_gain(y, left, right)
                if gain > best_gain:
                    best_gain = gain
                    best_split = (feature, threshold)

        if best_split is None:
            return {'class': np.bincount(y).argmax()}

        feature, threshold = best_split
        left_mask = X[:, feature] <= threshold
        right_mask = ~left_mask

        return {
            'feature': feature,
            'threshold': threshold,
            'left': self._build_tree(X[left_mask], y[left_mask], depth+1),
            'right': self._build_tree(X[right_mask], y[right_mask], depth+1)
        }

    def _predict_one(self, x, tree):
        if 'class' in tree:
            return tree['class']
        if x[tree['feature']] <= tree['threshold']:
            return self._predict_one(x, tree['left'])
        return self._predict_one(x, tree['right'])

    def predict(self, X):
        return [self._predict_one(x, self.tree) for x in X]


##3. Load and Split Iris Dataset

In [ ]:
iris = load_iris()
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


##4. Train and Evaluate Custom Decision Tree

In [ ]:
custom_tree = CustomDecisionTree(max_depth=3)
custom_tree.fit(X_train, y_train)

y_pred_custom = custom_tree.predict(X_test)
print("Custom Decision Tree Accuracy:", accuracy_score(y_test, y_pred_custom))

##5. Train and Evaluate Scikit-learn Decision Tree

In [ ]:
sk_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
sk_tree.fit(X_train, y_train)

y_pred_sk = sk_tree.predict(X_test)
print("Scikit-learn Decision Tree Accuracy:", accuracy_score(y_test, y_pred_sk))

##6. Ensemble Methods – Wine Dataset (Classification)

In [ ]:
wine = load_wine()
X, y = wine.data, wine.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

dt = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(random_state=42)

dt.fit(X_train, y_train)
rf.fit(X_train, y_train)

dt_f1 = f1_score(y_test, dt.predict(X_test), average='weighted')
rf_f1 = f1_score(y_test, rf.predict(X_test), average='weighted')

print("Decision Tree F1 Score:", dt_f1)
print("Random Forest F1 Score:", rf_f1)

##7. Hyperparameter Tuning – Random Forest (GridSearchCV)

In [ ]:
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

grid = GridSearchCV(RandomForestClassifier(random_state=42),
                    param_grid,
                    scoring='f1_weighted',
                    cv=5)

grid.fit(X_train, y_train)

print("Best Parameters:", grid.best_params_)
print("Best F1 Score:", grid.best_score_)

##8. Regression – Decision Tree & Random **Forest**

In [ ]:

X, y = wine.data, wine.target.astype(float)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

dt_reg = DecisionTreeRegressor(random_state=42)
rf_reg = RandomForestRegressor(random_state=42)

dt_reg.fit(X_train, y_train)
rf_reg.fit(X_train, y_train)

print("DT MSE:", mean_squared_error(y_test, dt_reg.predict(X_test)))
print("RF MSE:", mean_squared_error(y_test, rf_reg.predict(X_test)))

## 9. Hyperparameter Tuning – Random Forest Regressor (RandomizedSearchCV)

In [ ]:
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

random_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=5,
    cv=5,
    random_state=42
)

random_search.fit(X_train, y_train)

print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)
